# Perform Research with AutoGen

In [1]:
# import libraries

from lib_imports import *
import logging

import autogen
from autogen import UserProxyAgent, AssistantAgent

d:\Artificial Synapse Media\AutoGen_Research_Agents\aiagents_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load environment variables

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") 
AGENTOPS_API_KEY = os.getenv('AGENTOPS_API_KEY')



In [3]:
# log calls assigned to agents

logging.basicConfig(
    level=logging.DEBUG
) 

In [4]:
# initialize agentops
agentops.init(api_key=AGENTOPS_API_KEY, default_tags=["research-paper"])

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_session HTTP/11" 200 311
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=29f41ef5-89c3-4522-9e8e-b56df1d093b3


In [5]:
# research topic
topic = "Role of AI in Climate Modelling for Environmental Sustainability"

### AutoGen

In [6]:
# set configurations

config_list_gpt4 = [{"model": "gpt-4o", "api_key": OPENAI_API_KEY}]

gpt4_config = {
    "cache_seed": 42,  # change the cache_seed for different trials
    "temperature": 0,
    "config_list": config_list_gpt4,
    "timeout": 120,
}

## Construct Agents

### Agents Using AutoGen

In [7]:

user_proxy = UserProxyAgent(
    name="Admin",
    system_message="A human admin. Initialize the chat and pass on the instruction to the GenExplainer",
    code_execution_config=False,
)

genexplainer = AssistantAgent(
    name="GenExplainer",
    llm_config=gpt4_config,
    system_message="""GenExplainer. You collect research topic and break it down to subtopics, identifying key research 
                    questions or hypothesis. You also analyze the scope of the research topic and generate suitable 
                    research questions for an academic research. If the research topic is not good enough to generate 
                    enough scope for academic research, rephrase the topics and generate the research questions and hypothesis. 
                    You output the research topic, key research questions and hypothesis generated from the research topic.
                    """,
)

genreviewer = AssistantAgent(
    name="GenReviewer",
    llm_config=gpt4_config,
    system_message="""GenReviewer. You receive the research topic and subtopics from the GenExplainer Agent and then gather existing 
                    research, journal articles, papers, and other relevant literature from trusted sources. Summarize 
                    these findings, identify research gaps and organize the information. Produce this summary in the 
                    format of a standard academic literature review, highlighting important existing research and 
                    citing papers used in the review.
                    """,
)

genformulator = AssistantAgent(
    name="GenFormulator",
    system_message="""GenFormulator. You refine the research questions and subtopics from the GenExplainer Agent based on the 
                    Literature Review done by the GenReviewer and define the exact question the paper will address. If a 
                    hypothesis is required, propose one. Your output is the finalized research question and hypothesis.
                    """,
    llm_config=gpt4_config,
)

gendirector = AssistantAgent(
    name="GenDirector",
    llm_config=gpt4_config,
    system_message="""GenDirector. You receive the final research question from GenFormulator and literature review from 
                    GenReviewer and suggest appropriate research methods (qualitative, quantitative, or mixed) based on 
                    the research question. Also, outline data collection methods, sample sizes, tools, and techniques 
                    necessary for this research. Your output is the proposed research methodology, proposing necessary 
                    data collection methods, analysis plans, and tools for the research.
                    """,
)

gencollector = AssistantAgent(
    name="GenCollector",
    llm_config=gpt4_config,
    system_message="""GenCollector. You receive the proposed methodology from GenDirector Agent and gather real data from the 
                    web or simulate the data related to the research question. Ensure that the data are from verifiable 
                    and reliable sources, and are linked to the research being conducted. Your output are simulated or 
                    recommended datasets for analysis.
                    """,
)

genanalyst = AssistantAgent(
    name="GenAnalyst",
    system_message="""GenAnalyst. You receive data and analyze the collected data using appropriate statistical or 
                    qualitative analysis methods from GenCollector. For quantitative research, you could run statistical tests; for 
                    qualitative research, you would conduct thematic analysis. Your output are charts, graphs, 
                    insights, detected trends and predictions (if there are any). The analysis should be dynamic, 
                    customized to suit the requirement of the research field lies in.
                    """,
    llm_config=gpt4_config,
)

geninterpreter = AssistantAgent(
    name="GenInterpreter",
    llm_config=gpt4_config,
    system_message="""GenInterpreter. You collect data analysis results from GenAnalyst Agent and literature review from 
                    GenReviewer. You should interpret the analysis results, compares them with existing research in the 
                    review, and provides insights into how the findings answer the research question or support/refute 
                    the hypothesis, provided by the Formulator. Your output is a discussion of findings, implications, 
                    and potential limitations.
                    """,
)

genrecommender = AssistantAgent(
    name="GenRecommender",
    llm_config=gpt4_config,
    system_message="""GenRecommender. You collect discussion of findings, implications and potential limitations from the 
                    GenInterpreter Agent and summarize the research findings, proposes future research directions, and 
                    provides recommendations or solutions based on the findings. Your output is conclusion and 
                    recommendation section of the paper.
                    """,
)

genformatter = AssistantAgent(
    name="GenFormatter",
    system_message="""GenFormatter. You collect different parts of the research paper and ensure the entire research paper 
                    is formatted according to the chosen academic style (APA, MLA, etc.). Generate the fully formatted 
                    research paper using the final research topic from the GenFormulator Agent, the literature review 
                    from the GenReviewer Agent, the methodology from the GenDirector Agent, the data analysis from the 
                    GenAnalyst Agent, the discussion from the GenInterpreter Agent and the recommendation and conclusion
                    from the GenRecommender Agent. You also ensure proper citation process, collecting and formatting 
                    citations from literature sources. Your output is a fully formatted academic paper with citations 
                    and references.
                    """,
    llm_config=gpt4_config,
)

genqaengineer = AssistantAgent(
    name="GenQAEngineer",
    system_message="""GenQAEngineer. You collect a draft of an academic research paper from GenFormatter and perform proofreading, plagiarism 
                    checks, grammar and style analysis, and overall coherence evaluation. Your output is a PDF document 
                    containing a refined, high-quality research paper. 
                    """,
    llm_config=gpt4_config,
)

critic = AssistantAgent(
    name="Critic",
    system_message="Critic. Double check plan, claims, code from other agents and provide feedback. Check whether the plan includes adding verifiable info such as source URL.",
    llm_config=gpt4_config,
)

groupchat = autogen.GroupChat(
    agents=[user_proxy, genexplainer, genreviewer, genformulator, gendirector, gencollector, genanalyst, geninterpreter, genrecommender, genformatter, genqaengineer, critic], messages=[], max_round=3
)

manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=gpt4_config)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_agent HTTP/11" 200 9
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_agent HTTP/11" 200 9
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='d:\\Artificial Synapse Media\\AutoGen_Research_Agents\\aiagents_venv\\Lib\\site-packages\\certifi\\cacert.pem'
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_agent HTTP/11" 200 9
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_agent HTTP/11" 200 9
DEBUG:httpx:load_ssl_context verify=True cert=None tr

In [8]:

user_proxy.initiate_chat(
    manager,
    message="""Write an academic research paper on the topic: """ + topic,
)

Admin (to chat_manager):

Write an academic research paper on the topic: Role of AI in Climate Modelling for Environmental Sustainability

--------------------------------------------------------------------------------


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_events HTTP/11" 200 9
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_events HTTP/11" 200 9
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_events HTTP/11" 200 9
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_events HTTP/11" 200 9
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_events HTTP/11" 200 9
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.co

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_agent HTTP/11" 200 9
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='d:\\Artificial Synapse Media\\AutoGen_Research_Agents\\aiagents_venv\\Lib\\site-packages\\certifi\\cacert.pem'
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_agent HTTP/11" 200 9
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'content': "You are in a role play game. The following roles are available:\n                Admin: An attentive HUMAN user who can answer questions about the task, and can perform tasks such as running Python code or inputting command line commands at a Linux terminal and repo


Next speaker: GenExplainer



DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'content': 'GenExplainer. You collect research topic and break it down to subtopics, identifying key research \n                    questions or hypothesis. You also analyze the scope of the research topic and generate suitable \n                    research questions for an academic research. If the research topic is not good enough to generate \n                    enough scope for academic research, rephrase the topics and generate the research questions and hypothesis. \n                    You output the research topic, key research questions and hypothesis generated from the research topic.\n                    ', 'role': 'system'}, {'content': 'Write an academic research paper on the topic: Role of AI in Climate Modelling for Environmental Sustainability', 'name': 'Admin', 'role': 'user'}], 'model': 'gpt-4o', 'stream': False, 'temperature': 0}}
DEB

...

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_agent HTTP/11" 200 9
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='d:\\Artificial Synapse Media\\AutoGen_Research_Agents\\aiagents_venv\\Lib\\site-packages\\certifi\\cacert.pem'
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_agent HTTP/11" 200 9
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'content': "You are in a role play game. The following roles are available:\n                Admin: An attentive HUMAN user who can answer questions about the task, and can perform tasks such as running Python code or inputting command line commands at a Linux terminal and repo


Next speaker: GenReviewer



DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'content': 'GenReviewer. You receive the research topic and subtopics from the GenExplainer Agent and then gather existing \n                    research, journal articles, papers, and other relevant literature from trusted sources. Summarize \n                    these findings, identify research gaps and organize the information. Produce this summary in the \n                    format of a standard academic literature review, highlighting important existing research and \n                    citing papers used in the review.\n                    ', 'role': 'system'}, {'content': 'Write an academic research paper on the topic: Role of AI in Climate Modelling for Environmental Sustainability', 'name': 'Admin', 'role': 'user'}, {'content': '**Research Topic:** The Role of Artificial Intelligence in Climate Modelling for Environmental Sustainability\n\n**S

GenReviewer (to chat_manager):

**Literature Review: The Role of Artificial Intelligence in Climate Modelling for Environmental Sustainability**

**1. Overview of Climate Modelling**

Climate modelling is a critical tool in understanding and predicting climate change impacts. Traditional climate models, such as General Circulation Models (GCMs), simulate the Earth's climate system by solving complex mathematical equations that represent physical processes (Randall et al., 2007). These models have been instrumental in projecting future climate scenarios and informing policy decisions. However, they face limitations, including computational intensity, coarse spatial resolution, and uncertainties in parameterization (Flato et al., 2013).

**2. Introduction to Artificial Intelligence**

Artificial Intelligence (AI) encompasses a range of technologies that enable machines to perform tasks that typically require human intelligence, such as learning, reasoning, and problem-solving (Russell & 

ChatResult(chat_id=None, chat_history=[{'content': 'Write an academic research paper on the topic: Role of AI in Climate Modelling for Environmental Sustainability', 'role': 'assistant', 'name': 'Admin'}, {'content': '**Research Topic:** The Role of Artificial Intelligence in Climate Modelling for Environmental Sustainability\n\n**Scope of the Research Topic:**\nThe research topic explores the intersection of artificial intelligence (AI) and climate science, focusing on how AI technologies can enhance climate modelling to promote environmental sustainability. This involves examining the capabilities of AI in processing large datasets, improving the accuracy of climate predictions, and supporting decision-making processes for sustainable environmental practices.\n\n**Subtopics:**\n1. Overview of Climate Modelling\n   - Definition and importance of climate modelling\n   - Traditional methods of climate modelling\n   - Limitations of current climate models\n\n2. Introduction to Artificial

### Agents using OpenAI and AgentOps

In [9]:
from agent_classes import ExplainerAgent, LiteratureReviewer, FormulationAgent, MethodAgent, CollectorAgent
from agent_classes import DataAnalyst, DiscussionAgent, RecommendationAgent, CitationAgent, QAEngineer

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='d:\\Artificial Synapse Media\\AutoGen_Research_Agents\\aiagents_venv\\Lib\\site-packages\\certifi\\cacert.pem'


In [10]:
explainer = ExplainerAgent()
reviewer = LiteratureReviewer()
formulator = FormulationAgent()
director = MethodAgent()
collector = CollectorAgent()
analyst = DataAnalyst()
interpreter = DiscussionAgent()
recommender = RecommendationAgent()
formatter = CitationAgent()
qaengineer = QAEngineer()

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443


DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_agent HTTP/11" 200 9
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_agent HTTP/11" 200 9
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_agent HTTP/11" 200 9
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_agent HTTP/11" 200 9
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_agent HTTP/11" 200 9
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_agent HTTP/11" 200 9
DEBUG:urll

### Research Paper Generation

In [11]:
# Topic Understanding 

sub_topics = explainer.completion(
    "Generate research questions, subtopics and hypothesis for this topic : \n " + topic
)

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'Explainer. You collect research topic and break it down to subtopics, identifying key research \n                    questions or hypothesis. You also analyze the scope of the research topic and generate suitable \n                    research questions for an academic research. If the research topic is not good enough to generate \n                    enough scope for academic research, rephrase the topics and generate the research questions and hypothesis. \n                    You output the research topic, key research questions and hypothesis generated from the research topic.\n                    '}, {'role': 'user', 'content': 'Generate research questions, subtopics and hypothesis for this topic : \n Role of AI in Climate Modelling for Environmental Sustainability'}], 'model': 'gpt-4o', 'max_tokens': 16000, 'temperatur

DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x0000015D7BF7CF50>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x0000015D7D469490> server_hostname='api.openai.com' timeout=5.0
DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x0000015D7D43AC10>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Fri, 01 Nov 2024 17:51:40 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-contr

In [12]:
print(sub_topics)

**Research Topic:** Role of AI in Climate Modelling for Environmental Sustainability

**Scope of the Research Topic:**
The research topic explores how artificial intelligence (AI) can be integrated into climate modeling to enhance environmental sustainability. It examines the potential of AI to improve the accuracy, efficiency, and predictive capabilities of climate models, and how these improvements can contribute to sustainable environmental practices and policies.

**Subtopics:**

1. **AI Techniques in Climate Modelling:**
   - Machine learning algorithms and their application in climate data analysis.
   - Neural networks and their role in predicting climate patterns.
   - The use of AI in processing and interpreting satellite data for climate models.

2. **Improving Climate Model Accuracy:**
   - How AI can enhance the precision of climate predictions.
   - The role of AI in reducing uncertainties in climate models.
   - Case studies of AI-enhanced climate models and their outcome

In [13]:
# Literature Review

literature = reviewer.completion(
    "Write a literature review for these research questions: \n " + sub_topics
)

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'Reviewer. You receive the research topic and subtopics from the Explainer Agent and then gather existing \n                    research, journal articles, papers, and other relevant literature from trusted sources. Summarize \n                    these findings, identify research gaps and organize the information. Produce this summary in the \n                    format of a standard academic literature review, highlighting important existing research and \n                    citing papers used in the review.\n                    '}, {'role': 'user', 'content': 'Write a literature review for these research questions: \n **Research Topic:** Role of AI in Climate Modelling for Environmental Sustainability\n\n**Scope of the Research Topic:**\nThe research topic explores how artificial intelligence (AI) can be integrated into cl

DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x0000015D7D4A6910>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Fri, 01 Nov 2024 17:52:18 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-expose-headers', b'X-Request-ID'), (b'openai-organization', b'user-tb2wvztbcmilgcc878iv0tf5'), (b'openai-processing-ms', b'17968'), (b'openai-version', b'2020-10-01'), (b'x-ratelimit-limit-requests', b'500'), (b'x-ratelimit-limit-tokens', b'30000'), (b'x-ratelimit-remai

In [14]:
# Research Question Formulation

fin_question= formulator.completion(
    "Formulate a more concrete research question and hypothesis using the research topics: \n " + sub_topics + "\n  and the literature review: \n" + literature
)

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'Formulator. You refine the research questions and subtopics from the Explainer Agent based on the \n                    Literature Review done by the Reviewer and define the exact question the paper will address. If a \n                    hypothesis is required, propose one. Your output is the finalized research question and hypothesis.\n                    '}, {'role': 'user', 'content': "Formulate a more concrete research question and hypothesis using the research topics: \n **Research Topic:** Role of AI in Climate Modelling for Environmental Sustainability\n\n**Scope of the Research Topic:**\nThe research topic explores how artificial intelligence (AI) can be integrated into climate modeling to enhance environmental sustainability. It examines the potential of AI to improve the accuracy, efficiency, and predictive capabi

DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Fri, 01 Nov 2024 17:52:32 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-expose-headers', b'X-Request-ID'), (b'openai-organization', b'user-tb2wvztbcmilgcc878iv0tf5'), (b'openai-processing-ms', b'1386'), (b'openai-version', b'2020-10-01'), (b'x-ratelimit-limit-requests', b'500'), (b'x-ratelimit-limit-tokens', b'30000'), (b'x-ratelimit-remaining-requests', b'499'), (b'x-ratelimit-remaining-tokens', b'21828'), (b'x-ratelimit-reset-requests', b'120ms'), (b'x-ratelimit-reset-tokens', b'16.344s'), (b'x-request-id', b'req_fb2b826f145317d41460d08652592987'), (b'strict-transport-security', b'max-age=31536000; includeSubDomains; preload'), (b'CF-Cache-Status', b'DYNAMIC'), (b'X-Content-Type-Options', b'nosniff'), (b'Server', b'cloudflare'), (b'CF-RAY', b'8dbdbd2e7b0b9ff4-AMS'), (b'Content-Encoding', b'gz

In [15]:
# Research Methodology

methods = director.completion(
    "Suggest appropriate research methods using the final research question: \n " + fin_question + "\n  and the literature review: \n" + literature
)

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'Director. You receive the final research question from Formulator and literature review from \n                    Reviewer and suggest appropriate research methods (qualitative, quantitative, or mixed) based on \n                    the research question. Also, outline data collection methods, sample sizes, tools, and techniques \n                    necessary for this research. Your output is the proposed research methodology, proposing necessary \n                    data collection methods, analysis plans, and tools for the research.\n                    '}, {'role': 'user', 'content': "Suggest appropriate research methods using the final research question: \n **Finalized Research Question:**\n\nHow can the integration of artificial intelligence, specifically machine learning and neural networks, enhance the accuracy and 

DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x0000015D7BF39950>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x0000015D7D469490> server_hostname='api.openai.com' timeout=5.0
DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x0000015D7B922250>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Fri, 01 Nov 2024 17:53:09 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-contr

In [16]:
# Data Collection

data = collector.completion(
    "Gather necessary data using suggested research methodology: \n " + methods 
)

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'Collector. You receive the proposed methodology from Director Agent and gather real data from the \n                    web or simulate the data related to the research question. Ensure that the data are from verifiable \n                    and reliable sources, and are linked to the research being conducted. Your output are simulated or \n                    recommended datasets for analysis.\n                    '}, {'role': 'user', 'content': 'Gather necessary data using suggested research methodology: \n **Proposed Research Methodology**\n\nGiven the research question and the comprehensive literature review, a mixed-methods approach is most suitable for this study. This approach will allow for a robust analysis of both quantitative data from climate models and qualitative insights from policy implications and ethical con

DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x0000015D7BF208D0>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Fri, 01 Nov 2024 17:53:28 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-expose-headers', b'X-Request-ID'), (b'openai-organization', b'user-tb2wvztbcmilgcc878iv0tf5'), (b'openai-processing-ms', b'10428'), (b'openai-version', b'2020-10-01'), (b'x-ratelimit-limit-requests', b'500'), (b'x-ratelimit-limit-tokens', b'30000'), (b'x-ratelimit-remai

In [17]:
print(data)

To support the proposed research methodology, I will outline a plan for gathering both quantitative and qualitative data, including potential sources and simulated datasets where necessary.

### Quantitative Data Collection

1. **Historical Climate Data:**
   - **Source:** National Oceanic and Atmospheric Administration (NOAA), European Centre for Medium-Range Weather Forecasts (ECMWF), and NASA's Earth Observing System Data and Information System (EOSDIS).
   - **Data Description:** Historical records of temperature, precipitation, atmospheric pressure, and other climate variables spanning several decades.

2. **AI-Enhanced Climate Model Outputs:**
   - **Source:** Simulated data using AI models developed on platforms like TensorFlow or PyTorch.
   - **Data Description:** Outputs from AI-enhanced models that incorporate machine learning techniques to predict climate variables.

3. **Traditional Climate Model Outputs:**
   - **Source:** Intergovernmental Panel on Climate Change (IPCC) 

In [18]:
# Data Analysis

analysis = analyst.completion(
    "For academic research purposes, analyse this data: \n " + data 
)

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'Analyst. You receive data and analyze the collected data using appropriate statistical or \n                    qualitative analysis methods. For quantitative research, you could run statistical tests; for \n                    qualitative research, you would conduct thematic analysis. Your output are charts, graphs, \n                    insights, detected trends and predictions (if there are any). The analysis should be dynamic, \n                    customized to suit the requirement of the research field lies in.\n                    '}, {'role': 'user', 'content': "For academic research purposes, analyse this data: \n To support the proposed research methodology, I will outline a plan for gathering both quantitative and qualitative data, including potential sources and simulated datasets where necessary.\n\n### Quantitat

DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x0000015D7BFC99D0>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Fri, 01 Nov 2024 17:54:18 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-expose-headers', b'X-Request-ID'), (b'openai-organization', b'user-tb2wvztbcmilgcc878iv0tf5'), (b'openai-processing-ms', b'17011'), (b'openai-version', b'2020-10-01'), (b'x-ratelimit-limit-requests', b'500'), (b'x-ratelimit-limit-tokens', b'30000'), (b'x-ratelimit-remai

In [19]:
# Discussion Phase

discourse = interpreter.completion(
    "Write a discussion of findings and possible implications using the data analysis: \n " + analysis + "\n  and the literature review: \n" + literature
)

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'Interpreter. You collect data analysis results from Analyst Agent and literature review from \n                    Reviewer. You should interpret the analysis results, compares them with existing research in the \n                    review, and provides insights into how the findings answer the research question or support/refute \n                    the hypothesis, provided by the Formulator. Your output is a discussion of findings, implications, \n                    and potential limitations.\n                    '}, {'role': 'user', 'content': "Write a discussion of findings and possible implications using the data analysis: \n To analyze the proposed research methodology, we will focus on both quantitative and qualitative data aspects, utilizing statistical and thematic analysis methods. Here's a breakdown of the analy

DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x0000015D7D4F02D0>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Fri, 01 Nov 2024 17:54:35 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-expose-headers', b'X-Request-ID'), (b'openai-organization', b'user-tb2wvztbcmilgcc878iv0tf5'), (b'openai-processing-ms', b'9073'), (b'openai-version', b'2020-10-01'), (b'x-ratelimit-limit-requests', b'500'), (b'x-ratelimit-limit-tokens', b'30000'), (b'x-ratelimit-remain

In [20]:
# Conclusion and Recommendations

conclusion = recommender.completion(
    "Write a recommendation and coclusion using the discussion generated from the academic research: \n " + discourse
)

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'Recommender. You collect discussion of findings, implications and potential limitations from the \n                    Interpreter Agent and summarize the research findings, proposes future research directions, and \n                    provides recommendations or solutions based on the findings. Your output is conclusion and \n                    recommendation section of the paper.\n                    '}, {'role': 'user', 'content': "Write a recommendation and coclusion using the discussion generated from the academic research: \n ### Discussion of Findings\n\nThe analysis of both quantitative and qualitative data provides a comprehensive understanding of the role of AI in climate modeling and its implications for environmental policy. The findings from the quantitative analysis reveal that AI-enhanced climate models signi

DEBUG:openai._base_client:Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=5.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x0000015D7BF52D50>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x0000015D7D469490> server_hostname='api.openai.com' timeout=5.0
DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x0000015D7D4C5310>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:http

In [21]:
# Formatting and Citation

draft =formatter.completion(
    "Generate the fully formatted research paper using the final research topic: \n " + fin_question + "\n the literature review:" + literature
    + "\n the methodology:" + methods + "\n the data analysis:" + analysis + "\n the discussion:" + discourse + "  \n and the recommendation and conclusion:" + conclusion 
)
 


DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'Formatter. You collect different parts of the research paper and ensure the entire research paper \n                    is formatted according to the chosen academic style (APA, MLA, etc.). You also ensure proper \n                    citation process, collecting and formatting citations from literature sources. Your output is a \n                    fully formatted academic paper with citations and references.\n                    '}, {'role': 'user', 'content': "Generate the fully formatted research paper using the final research topic: \n **Finalized Research Question:**\n\nHow can the integration of artificial intelligence, specifically machine learning and neural networks, enhance the accuracy and predictive capabilities of climate models, and what impact does this have on the development of sustainable environmental pol

DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x0000015D7BD52D90>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Fri, 01 Nov 2024 17:55:51 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-expose-headers', b'X-Request-ID'), (b'openai-organization', b'user-tb2wvztbcmilgcc878iv0tf5'), (b'openai-processing-ms', b'39209'), (b'openai-version', b'2020-10-01'), (b'x-ratelimit-limit-requests', b'500'), (b'x-ratelimit-limit-tokens', b'30000'), (b'x-ratelimit-remai

In [22]:
# Complete Research Paper

paper =qaengineer.completion(
    "Do the final touch up of the draft: \n " + draft + "output a PDF format of the doocument"
)

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'QAEngineer. You collect a draft of an academic research paper and perform proofreading, plagiarism \n                    checks, grammar and style analysis, and overall coherence evaluation. Your output is a PDF document \n                    containing a refined, high-quality research paper. \n                    '}, {'role': 'user', 'content': "Do the final touch up of the draft: \n # Enhancing Climate Models with Artificial Intelligence: Implications for Environmental Policy\n\n## Abstract\n\nThis research explores the integration of artificial intelligence (AI), specifically machine learning and neural networks, into climate modeling to enhance accuracy and predictive capabilities. The study examines the impact of these advancements on the development of sustainable environmental policies. The hypothesis posits that AI si

DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Fri, 01 Nov 2024 17:56:37 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-expose-headers', b'X-Request-ID'), (b'openai-organization', b'user-tb2wvztbcmilgcc878iv0tf5'), (b'openai-processing-ms', b'35862'), (b'openai-version', b'2020-10-01'), (b'x-ratelimit-limit-requests', b'500'), (b'x-ratelimit-limit-tokens', b'30000'), (b'x-ratelimit-remaining-requests', b'499'), (b'x-ratelimit-remaining-tokens', b'21828'), (b'x-ratelimit-reset-requests', b'120ms'), (b'x-ratelimit-reset-tokens', b'16.344s'), (b'x-request-id', b'req_644ff80414ee9c0d0b029d9

In [23]:
with open("climatemodelling.md", "w") as file: file.write(paper)

In [24]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

def save_to_pdf(text, filename):
    c = canvas.Canvas(filename, pagesize=letter)
    width, height = letter
    for line in text.strip().splitlines():
        height -= 11
        c.drawString(10, height, line.strip())
    c.save()

save_to_pdf(paper, "research.pdf")

In [25]:
agentops.end_session("Success")

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/update_session HTTP/11" 200 140
🖇 AgentOps: Session Stats - Duration: 12m 34.9s | Cost: $0.149575 | LLMs: 18 | Tools: 0 | Actions: 59 | Errors: 0
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=29f41ef5-89c3-4522-9e8e-b56df1d093b3
